In [1]:
import numpy as np
import pandas as pd
import orca
import os; os.chdir('../')
import warnings; warnings.simplefilter('ignore')
from urbansim.utils import misc
import pandana as pdna
    
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
import pandas as pd
import orca as orc
# import os; os.chdir('/home/juan/ual_model_workspace/spring-2019-models/')
import warnings; warnings.simplefilter('ignore')
from matplotlib import pyplot as plt
import matplotlib.animation as animation
import seaborn as sns

# from scripts import datasources, models, variables, utils
# scripts = '~/ual_model_workspace/spring-2019-models/scripts'

In [2]:
from scripts import datasources, models, variables, utils

Registering model step 'auto_ownership'
Registering model step 'TOD_choice'
Registering model step 'primary_mode_choice'
Registering model step 'WLCM'


In [3]:
import urbansim_templates
urbansim_templates.__version__

'0.1.1'

# Loading data

In [4]:
orca.run(['initialize_network_small', 'initialize_network_walk','impute_missing_skims']) 

Running step 'initialize_network_small'
Time to execute step 'initialize_network_small': 0.00 s
Running step 'initialize_network_walk'
Time to execute step 'initialize_network_walk': 0.00 s
Running step 'impute_missing_skims'
Time to execute step 'impute_missing_skims': 95.46 s
Total time to execute iteration 1 with iteration value None: 95.46 s


In [5]:
#Loading Data 
beam_skims = orca.get_table('beam_skims').to_frame()
students = pd.read_csv('/home/juan/ual_model_workspace/spring-2019-models/notebooks-juan/students_with_school_id.csv')
schools = pd.read_csv('/home/juan/ual_model_workspace/spring-2019-models/notebooks-juan/schools.csv')
parcels = orca.get_table('parcels').to_frame()

#Preprocessing
students = students[students.AGE <= 18]

In [6]:
# schools.head()

In [7]:
#Adding list of grades offered by each school
list_grades = []
for index, row in schools.loc[:,schools.columns.str.startswith("grade_")].iterrows():
    x = np.array(row)
    list_grades.append(x)
    
schools['list_grades'] = list_grades

## Defining set choice contrains

In [8]:
#Small netwrok 
nodessmall = pd.read_csv('/home/data/fall_2018/bay_area_tertiary_strongly_nodes.csv').set_index('osmid')
edgessmall = pd.read_csv('/home/data/fall_2018/bay_area_tertiary_strongly_edges.csv')
# netsmall = pdna.Network(nodessmall.x, nodessmall.y, edgessmall.u,
#                                 edgessmall.v, edgessmall[['length']],
#                                 twoway=False)
# netsmall.precompute(25000)



In [10]:
def node_id_small(x, y, netsmall):
    """ Return the node ID given a pair of coordinates"""
    idssmall = netsmall.get_node_ids(x, y)
    return idssmall

# schools['nodeID'] = node_id_small(schools, netsmall)

In [38]:
# students.columns[-100:]
schools.columns[-100:]

Index(['CDSCode', 'School', 'District', 'County', 'Street', 'City', 'Zip',
       'State', 'Latitude', 'Longitude', 'type', 'grade_0', 'grade_1',
       'grade_2', 'grade_3', 'grade_4', 'grade_5', 'grade_6', 'grade_7',
       'grade_8', 'grade_9', 'grade_10', 'grade_11', 'grade_12',
       'Kindergarten Enrollment', 'Grade 1 Enrollment', 'Grade 2 Enrollment',
       'Grade 3 Enrollment', 'Grade 4 Enrollment', 'Grade 5 Enrollment',
       'Grade 6 Enrollment', 'Grade 7 Enrollment', 'Grade 8 Enrollment',
       'Ungraded Elementary Enrollmnet', 'Grade 9 Enrollment',
       'Grade 10 Enrollment', 'Grade 11 Enrollment', 'Grade 12 Enrollment',
       'Ungraded Secondary Enrollment', 'Total Enrollment', 'sw_rank_2108',
       'sw_rank_2017', 'sw_rank_2016', 'ss_rank_2018_', 'ss_rank_2017',
       'ss_rank_2016', 'nodeID', 'parcel_id', 'school_id', 'list_grades'],
      dtype='object')

In [11]:
#Merging students and schools 
students_1 = students.merge(schools, how = 'left', on = 'school_id')

# Droping students with no assigned school
students_1 = students_1.dropna(subset=['school_id'])

In [16]:
students_1.columns[-100:]


Index(['WSTFIP_persons', 'W2PrimaryCity_persons', 'W2STFIP_persons',
       'SPrimaryCity_persons', 'SSTFIP_persons', 'PERWGT', 'EXPPERWGT',
       'WNAME_lookup', 'WCITY_lookup', 'WZIP_lookup', 'WXST1_lookup',
       'WXST2_lookup', 'WXCORD_lookup', 'WYCORD_lookup', 'WNAME2_lookup',
       'WCITY2_lookup', 'WZIP2_lookup', 'WXST2_1_lookup', 'WXST2_2_lookup',
       'SNAME_lookup', 'SCITY_lookup', 'SZIP_lookup', 'SXST1_lookup',
       'SXST2_lookup', 'SXCORD_lookup', 'SYCORD_lookup', 'WCTFIP_lookup',
       'WTRACT_lookup', 'WBLOCK', 'WPrimaryCity_lookup', 'WSTFIP_lookup',
       'W2PrimaryCity_lookup', 'W2STFIP_lookup', 'SCTFIP_lookup',
       'STRACT_lookup', 'SBLOCK', 'SPrimaryCity_lookup', 'SSTFIP_lookup',
       'HCITY', 'HZIP', 'HXCORD', 'HYCORD', 'HBLOCK', 'HTRACT', 'HCTFIP',
       'HPrimaryCity', 'parcel_id_home', 'parcel_id_work', 'zone_id_home',
       'zone_id_work', 'school_id', 'CDSCode', 'School', 'District', 'County',
       'Street', 'City', 'Zip', 'State', 'Latitude', 

In [12]:
#Define the node ID for each home and school location in students dataset
students_1['node_id_home'] = node_id_small(students_1.HXCORD, students_1.HYCORD, netsmall)
students_1['node_id_school'] = node_id_small(students_1.Longitude, students_1.Latitude, netsmall)

#Creating a df for public shcools only
df_public = students_1[students_1.school_id <= 2827].loc[:,['SAMPN', 'PERNO', 'school_id',
                                                        'AGE','HCITY','HYCORD','HXCORD',
                                                        'SNAME_lookup','SCITY_lookup', 
                                                        'node_id_home', 'node_id_school']]

# #Creating a df for private shcools only
df_private = students_1[students_1.school_id > 2827].loc[:,['SAMPN', 'PERNO', 'school_id',
                                                        'AGE','HCITY','HYCORD','HXCORD',
                                                        'SNAME_lookup','SCITY_lookup', 
                                                        'node_id_home', 'node_id_school']]

In [13]:
df_public.head(50)

,SAMPN,PERNO,school_id,AGE,HCITY,HYCORD,HXCORD,SNAME_lookup,SCITY_lookup,node_id_home,node_id_school
4,1051795,3,1919.0,13,SAN FRANCISCO,37.757638,-122.484605,presidio,san francisco,258912606,258910950
5,1059276,2,1710.0,15,NOVATO,38.046522,-122.523075,novato high,novato,345571129,2807230972
6,1173970,3,1712.0,8,NOVATO,38.045329,-122.524101,hamilton elementary,novato,345571129,345571467
7,1173970,4,1712.0,6,NOVATO,38.045329,-122.524101,hamilton elementary,novato,345571129,345571467
8,1064644,3,2089.0,15,DALY CITY,37.692202,-122.485125,capatino high,san bruno,65423908,420520650
9,1088852,3,1534.0,11,WALNUT CREEK,37.915146,-122.013320,walnut acres,walnut creek,1793901645,5243955958
10,1088852,4,1534.0,9,WALNUT CREEK,37.915146,-122.013320,walnut acres,walnut creek,1793901645,5243955958
11,1089378,2,1361.0,8,SAN LEANDRO,37.686076,-122.136884,del ray elementary,san lorenzo,4182144711,53026754
12,1095531,3,1901.0,6,SAN FRANCISCO,37.741154,-122.426453,starr king,san francisco,318299418,65320199
13,1119016,3,2409.0,15,PALO ALTO,37.443377,-122.153627,paly high,palo alto,463052554,289661267


In [14]:
#Setting public and private schools as POIs. 
netsmall.set_pois('public_school', 500000, 10000, schools[schools.type == 'public']["Longitude"], 
                  schools[schools.type == 'public']["Latitude"])


netsmall.set_pois('private_school', 500000, 10000, 
                  schools[schools.type == 'private'].Longitude, 
                  schools[schools.type == 'private'].Latitude)

In [15]:
netsmall.set_pois('public_school', 500000, 10000, df_public['HXCORD'], df_public["HYCORD"])

In [16]:
#Public schools. 95% of the time, the school is within the x's closest schools. Finding x
n =50

# n closest public schools per each node in netsmall
distance_matrix_public = netsmall.nearest_pois(200000, 
                                               'public_school', 
                                               num_pois=n,
                                               include_poi_ids=True)

# Selects POIS id's only 
public_nodes = distance_matrix_public.iloc[:,n:]

# Creates a list of n closest POIs for each node
list_values = []
for index, row in public_nodes.iterrows():
    x = np.array(row)
    x = x[~np.isnan(x)]
    list_values.append(x)

#Add created list to public nodes 
public_nodes['list_values'] = list_values


merge = df_public.merge(public_nodes.loc[:,['list_values']], how = 'left', left_on = 'node_id_home', right_index= True)

school_position = []
for index, row in merge.iterrows():
    school_position.append(np.isin(row['school_id']-1000,row['list_values'])==True)
    
merge['school_position'] = school_position

merge.school_position.mean()

0.01969057665260197

In [17]:
#Private schools. 95% of the time, the school is within the x's closest schools. Finding x
n =100

distance_matrix_private = netsmall.nearest_pois(100000, 
                                                'private_school', 
                                                num_pois=n, 
                                                include_poi_ids=True)

private_nodes = distance_matrix_private.iloc[:,n:]

list_values = []
for index, row in private_nodes.iterrows():
    x = np.array(row)
    x = x[~np.isnan(x)]
    list_values.append(x)
    
private_nodes['list_values'] = list_values


merge = df_private.merge(private_nodes.loc[:,['list_values']], how = 'left', left_on = 'node_id_home', right_index= True)

school_position = []
for index, row in merge.iterrows():
    school_position.append(np.isin(row['school_id']-1000,row['list_values'])==True)
    
merge['school_position'] = school_position

merge.school_position.mean()

0.8758465011286681

In [ ]:
# students = students[students.AGE <= 18]

In [18]:
def school_available(age, grades_offered):
    """ Checks if a school offers a grade according to the age range
    Input: Age: int in range (5-18)
           grades_offered: 13 element array
    Output: True if grade if offered acoording to age, false otherwise. 
     """
    index = age - 6
    
    if index < 0:
        index = 0

    if index < 12:
        result = (grades_offered[index] == 1) | (grades_offered[index + 1] == 1)
        
    elif index == 12:
        result = (grades_offered[index] == 1)
    
    return result

In [19]:
def school_choice_set(house_node_id, kid_age):
    """ Determines the school choice set given home node id and age of the student (4-18)
    Output: Pandas series with available school IDs"""
    
    public_id = distance_matrix_public.iloc[:,50:].loc[house_node_id] + 1000
    private_id = distance_matrix_private.iloc[:,100:].loc[house_node_id] + 1000
    schools_filter = pd.concat([public_id, private_id])
    schools_set = schools[schools.school_id.isin(schools_filter)]
    school_availability = [school_available(kid_age, x) for x in schools_set.list_grades]
    schools_available = schools_set[school_availability].school_id
    
    return schools_available

In [20]:
school_choice_set(5418694673, 5)


34      1034
429     1429
687     1687
921     1921
1051    2051
1068    2068
1073    2073
1074    2074
1128    2128
1199    2199
1380    2380
1514    2514
1625    2625
1727    2727
1782    2782
1783    2783
1784    2784
1794    2794
1795    2795
2207    3207
2208    3208
2228    3228
2229    3229
2230    3230
2231    3231
2232    3232
2233    3233
2239    3239
2242    3242
2243    3243
        ... 
2289    3289
2290    3290
2291    3291
2292    3292
2293    3293
2294    3294
2295    3295
2296    3296
2297    3297
2298    3298
2299    3299
2300    3300
2302    3302
2306    3306
2311    3311
2316    3316
2324    3324
2329    3329
2337    3337
2342    3342
2349    3349
2359    3359
2360    3360
2362    3362
2363    3363
2365    3365
2373    3373
2429    3429
2455    3455
2465    3465
Name: school_id, Length: 97, dtype: int64

# Additional code

In [ ]:
# #Creating school availability matrix based on distance
# school_availability_distance = pd.DataFrame(nodessmall.index)
# for x in range(1000,3547,1):
#     name = "school_" + str(x)
#     school_availability_distance[name] = x
# school_availability_distance = school_availability_distance.set_index('osmid')

In [ ]:

a = students.sample(10)
a.loc[:,['AGE','HCITY','HYCORD','HXCORD', 'school_id']]

In [ ]:
students[(students.SAMPN == 7176467) & (students.PERNO ==4) ][['HCITY', 'HYCORD','HXCORD',
                                                                'AGE','SNAME_lookup','SCITY_lookup', 'school_id']]

In [ ]:
schools[schools.school_id == 1271]

In [ ]:
d[d.type == 'public'].sort_values('dist', ascending=False)

In [ ]:
# os.listdir("/home/data/fall_2018/")

In [ ]:
# # Estimating 95th percentail of distances between home and school
# a = schools.merge(parcels[['zone_id']], left_on ='parcel_id', right_index=True)
# b = students[['SAMPN','PERNO','zone_id_home', 'school_id']].merge(a[['zone_id','school_id','type']], on = 'school_id')
# c = beam_skims.reset_index(level=['from_zone_id', 'to_zone_id'])
# d = b.merge(c.loc[:,['from_zone_id','to_zone_id', 'dist']], 
#             left_on = ['zone_id_home','zone_id'], 
#             right_on = ['from_zone_id', 'to_zone_id'])

# public_dist_limit = d[d.type == 'public'].quantile(1).dist
# private_dist_limit = d[d.type == 'private'].quantile(1).dist

# sns.distplot(d[d.type == 'public'].dist,kde=False)
# sns.distplot(d[d.type == 'private'].dist,kde=False);
# d.sort_values('dist', ascending=False)

In [ ]:
# a = students.loc[:,['HCITY','SNAME_lookup', 'SCITY_lookup', 'SZIP_lookup',
#                 'SXCORD_lookup', 'SYCORD_lookup', 'SPrimaryCity_lookup',]]
# same_city = []
# for index, row in a.iterrows():
#     if row['HCITY'].lower() == row['SCITY_lookup']:
#         same_city.append(True)
#     else:
#         same_city.append(False)

# a['same_city'] = same_city
# a.same_city.mean()

In [ ]:
#'network_aggregations_small'])
# orca.run(['initialize_network_walk', 'network_aggregations_walk'])

# orca.get_table('nodeswalk').to_frame().to_csv('./data/walk_net_vars.csv')
# orca.get_table('nodessmall').to_frame().to_csv('./data/drive_net_vars.csv')
# orca.get_table('zones').to_frame().to_csv('./data/zones_w_access_vars-2025-b-lt.csv')

In [ ]:
# os.listdir("/home/data/spring_2019/2025")

In [ ]:
# a = students.loc[:,['SAMPN', 'PERNO', 'AGE', 'HPrimaryCity','school_id']].merge(schools.loc[:,['school_id','City','District']], how = 'inner', on = 'school_id')
# students[students.SAMPN == 1342402].loc[:,['SNAME_lookup','school_id', 'HPrimaryCity']]
# schools[schools.school_id == 1710]